# The Backtranslation Notebook to Rule Over Them All
Please do ignore the title, I just wanted to make it sound cool. This notebook is a simple demonstration of how to use backtranslation to improve the performance of a machine learning model. The notebook is divided into the following sections:
1. Introduction and Simple Generation
2. Applying Data Preparation/Filtering
3. Investigating Iterative Backtranslation
4. Applying a similar pipeline to a more complex model (ALMA-R)
5. Conclusion

Results:

| Model | IT |  | NEWS | |
| ---   |---- | --- | --- | --- |
|       | BLEU| COMET|BLEU|COMET|
| Base | 38.31 (0.819±.145) | 29.226±7.710 (0.834±0.077) |
| FT on IT-parallel | 38.92 (0.825±.145) | 28.646 (0.827±0.064) |
| FT on BT | 41.827 (0.84±.0) | 27.115±7.197 (0.820±0.084) |
| FT on IT-parallel and BT | 39.879 (0.86±.1) | 26.655±7.249 (0.819±0.084) |

In [1]:
# First cell in the notebook to enable autoreload of modules
%load_ext autoreload
%autoreload 2

In [88]:
from pathlib import Path
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
from utils.data import TranslationDataModule
from utils.models import TranslationLightning
from pytorch_lightning import Trainer
from peft import LoraConfig, get_peft_model
# Isn't it so nice and clean now? I went through FOUR different ways of doing this before I thought of this one. WDWFDGFEQWDQWFGA

SRC = "de"
TGT = "en"
BATCH_SIZE = 16
MAX_LENGTH = 128

cwd = Path.cwd()
data_dir = cwd / "Data"

it_parallel = "it-parallel"
news_dataset = "train-euro-news-big"
it_mono = "it-mono"

test_folder = cwd / "tests"

mname = f"facebook/wmt19-{SRC}-{TGT}"
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)

config = LoraConfig(
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0.2,
    target_modules = ["v_proj", "q_proj"]
)

it_parallel_data = TranslationDataModule(data_dir / it_parallel, SRC, TGT, tokenizer, None, batch_size=BATCH_SIZE, max_length=MAX_LENGTH)
# Add news_dataset


model = get_peft_model(model, config)
model.print_trainable_parameters()

model_pl = TranslationLightning(model, tokenizer, lr=3e-4, adam_beta=(0.9, 0.98), weight_decay=1e-4, test_folder = test_folder)

trainer = Trainer(max_steps=10,  gradient_clip_val=0.1, val_check_interval = 0.25, limit_val_batches=0.25)

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-de-en and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,179,648 || all params: 273,027,072 || trainable%: 0.4320626490841172


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [153]:
it_parallel_data = TranslationDataModule(data_dir / it_parallel, SRC, TGT, tokenizer, model, batch_size=BATCH_SIZE, max_length=MAX_LENGTH)

it_parallel_data.setup('fit')
dataset = it_parallel_data.train

In [155]:
batch = next(iter(it_parallel_data.test_dataloader()))
# batch['labels'][batch['labels']==1] = -100
# batch.pop('decoder_input_ids')
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])


In [156]:
model_pl.training_step(batch, 0)

tensor(0.9212, grad_fn=<NllLossBackward0>)

In [4]:
it_parallel_data.setup('fit')
# batch = next(iter(it_parallel_data.train_dataloader()))

# print(batch["labels"].)

## Simple Generation
Here we simply load a pre-trained model and run inference on the test set. The model loaded is a pre-trained model from the [Hugging Face Transformers](https://huggingface.co/transformers/) library. 


In [3]:
# IT-parallel
results = trainer.predict(model_pl, datamodule=it_parallel_data)
bleu = sum(r[2]["score"] for r in results) / len(results)
print("Average BLEU:", bleu)

# News corpus
# ...

In [ ]:
# Now we can train the model
trainer.fit(model_pl, datamodule=it_parallel_data)

# IT-parallel
trainer.predict(model_pl, datamodule=it_parallel_data)
bleu = sum(r[2]["score"] for r in results) / len(results)
print("Average BLEU:", bleu)

# News corpus

### Now the key ingredient: Backtranslation
Backtranslation uses a reverse model to generate synthetic data. This synthetic data is then used to train the model. The idea is that the synthetic data will help the model generalize better.

In [ ]:
reverse_model = FSMTForConditionalGeneration.from_pretrained(f"facebook/wmt19-{TGT}-{SRC}")
reverse_tokenizer = FSMTTokenizer.from_pretrained(f"facebook/wmt19-{TGT}-{SRC}")

it_mono = TranslationDataModule(data_dir / it_mono, TGT, SRC, reverse_tokenizer, batch_size=BATCH_SIZE, max_length=MAX_LENGTH)

output_folder = test_folder / f"generation-{TGT}-{SRC}"
reverse_lightning = TranslationLightning(reverse_model, reverse_tokenizer, lr=3e-4, adam_beta=(0.9, 0.98), weight_decay=1e-4, test_folder = output_folder)

trainer.predict(reverse_lightning, it_mono.train_dataloader())

Now that we have generated the data, we can now copy in the generated data.

In [ ]:
train_file = output_folder / "hypothesis.hyp"
train_file_target = output_folder / "source.src"
train_file.rename(data_dir / f"generated-{it_mono}" / f"train.{SRC}")
train_file_target.rename(data_dir / f"generated-{it_mono}" / f"train.{TGT}")

import shutil
bt_dir = data_dir / f"generated-{it_mono}"
shutil.copy(data_dir / it_mono / f"dev.{SRC}", bt_dir / f"dev.{SRC}")
shutil.copy(data_dir / it_mono / f"dev.{TGT}", bt_dir / f"dev.{TGT}")
shutil.copy(data_dir / it_mono / f"test.{SRC}", bt_dir / f"test.{SRC}")
shutil.copy(data_dir / it_mono / f"test.{TGT}", bt_dir / f"test.{TGT}")


In [ ]:
# And now we can finally train...
it_mono_bt = TranslationDataModule(data_dir / f"generated-{it_mono}", SRC, TGT, tokenizer, batch_size=BATCH_SIZE, max_length=MAX_LENGTH)
trainer.fit(model_pl, datamodule=it_mono_bt)

In [ ]:
model_pl.test_folder = test_folder / f"bt_{it_mono}"
# IT-parallel
results = trainer.predict(model_pl, datamodule=it_parallel_data)
print("IT-parallel", sum(r[2]["score"] for r in results) / len(results))
# News corpus
# ...

And now finally we can concatenate both the original and the generated data and train the model on it.

In [ ]:
parallel_plus_bt = data_dir / f"parallel+bt_generated_{it_mono}"
shutil.copytree(bt_dir, parallel_plus_bt)
train_file = parallel_plus_bt /f"train.{SRC}"
with open(train_file, "r+") as f:
    with open(data_dir / it_parallel / f"train.{SRC}", "r") as f2:
        f.write("\n" + f2.read())
        
train_file_target = parallel_plus_bt / f"train.{TGT}"
with open(train_file_target, "r+") as f:
    with open(data_dir / it_parallel / f"train.{TGT}", "r") as f2:
        f.write("\n" + f2.read())


In [ ]:
# Fitting the model on the combined data
parallel_plus_bt_data = TranslationDataModule(parallel_plus_bt, SRC, TGT, tokenizer, batch_size=BATCH_SIZE, max_length=MAX_LENGTH)
trainer.fit(model_pl, datamodule=parallel_plus_bt_data)

In [ ]:
# IT-parallel
results = trainer.predict(model_pl, datamodule=it_parallel_data)
print("IT-parallel BLEU:", sum(r[2]["score"] for r in results) / len(results))
# News corpus
# ...

## 3. Investigating Iterative Backtranslation
The basic idea, proposed by Hoang et al. (2018), is to iteratively backtranslate the data. For our purposes, we require two separate models, one for forward translation (EN-DE) and one for reverse translation (DE-EN). We can then use them in turns, first generating synthetic data, training the model, and then using the trained forward model to generate synthetic data for the reverse model. This process may be iterated until convergence.

We can combine this approach with concatenating the high quality parallel data to improve the performance of both models.

In [ ]:
# First we might want to fine-tune the model on the IT-parallel data
!python train.py \
    --output_dir tests/reverse-it-parallel-de-en \
    --srclang de --tgtlang en \
    --batch_size 32 \
    --max_length 128 \
    --epochs 15 \
    --train_dir it-parallel --seed 42 \
    --check_val_every_n_epoch 5

In [ ]:
# Now we can generate translations for the IT-mono data
!python generate.py \
    --output_dir tests/reverse-generation-de-en \
    --srclang de --tgtlang en \
    --load_from_checkpoint /home/mkrastev/nlp2/Models/it-mono+parallel/checkpoints/epoch=14-step=56250.ckpt \
    --batch_size 32 \
    --max_length 128 \
    --data_dir Data/it-mono --split train

In [ ]:
# And now we can fine-tune the model on the generated data
!python train.py \
    --output_dir tests/reverse-generation-it-mono-de-en \
    --srclang de --tgtlang en \
    --batch_size 32 \
    --max_length 128 \
    --epochs 15 \
    --train_dir it-mono --seed 42 \
    --check_val_every_n_epoch 5

In [ ]:
# Then generate translations for the IT-mono data and the forward model
!python generate.py \
    --output_dir tests/iterative-generation-it-mono-en-de \
    --srclang en --tgtlang de \
    --load_from_checkpoint 
    --batch_size 32 \
    --max_length 128 \
    --data_dir it-mono

In [ ]:
# And finally retrain fine-tune the model on the combined data
!python train.py \
    --output_dir tests/iterative-generation-it-mono-en-de \
    --srclang en --tgtlang de \
    --batch_size 32 \
    --max_length 128 \
    --epochs 15 \
    --train_dir iterative-it-mono --seed 42 \
    --check_val_every_n_epoch 5

And done! We have now successfully implemented iterative backtranslation. We can repeat this process until convergence (or until we run out of cloud compute, GPUs aren't cheap!).